# 01_make_dataset_frontback_yoloseg - データセット生成ノートブック

## 概要
YOLO-Segをファインチューニングする用のデータセットを生成

## 処理フロー（概要）
1. COCOアノテーション読み込み
2. **キーポイントゲート（IDレベル）**：全身が十分に映っている歩行者がいる画像のみ抽出
3. ゲート通過IDから `train` / `test` / `val` を作成（画像はまだダウンロードしない）
4. 通過画像のみダウンロード
5. MEBOWで向き推定 → SAME/OPS分類
6. COCOセグメンテーション → YOLO-Seg形式ラベル（ポリゴン）へ変換・エクスポート
7. `data.yaml` 生成

## クラス定義
- `0: SAME`：カメラと同方向（角度 $\le 45^\circ$ または $\ge 315^\circ$）
- `1: OPS`：それ以外（逆方向）

## キーポイントゲート
- 必須：両肩・両ひざ（可視性 $v \ge 1$）
- 画像内に **1人でも** 条件を満たす歩行者がいれば → その画像の全personをラベル化
- 誰も満たさなければ → 画像はダウンロードせずスキップ

## 1. 環境確認とパス設定

プロジェクトルートパスを設定し、GPU利用可否を確認します。

In [ ]:
# ============================================================================
# パス設定と環境確認
# ============================================================================
import sys
from pathlib import Path
import os
import shutil
import random

# ----- プロジェクトルートの設定 -----
# scripts/ から1つ上のディレクトリがプロジェクトルート
PROJECT_ROOT = Path("..").resolve()
print(f"PROJECT_ROOT: {PROJECT_ROOT}")

# ----- 各ディレクトリパスの定義 -----
DATA_DIR = PROJECT_ROOT / "data"
DATA_RAW = DATA_DIR / "raw"                           # COCO画像DL先
DATA_OUT = DATA_DIR / "dataset_frontback_yoloseg"     # 出力先

# COCOアノテーション
COCO_ANN_DIR = DATA_DIR / "annotations"
ANN_TRAIN = COCO_ANN_DIR / "instances_train2017.json"
ANN_VAL   = COCO_ANN_DIR / "instances_val2017.json"

# COCOキーポイント（ノイズフィルタ用）
KP_TRAIN = COCO_ANN_DIR / "person_keypoints_train2017.json"
KP_VAL   = COCO_ANN_DIR / "person_keypoints_val2017.json"

# MEBOWモデル（同一リポジトリ配下）
MEBOW_ROOT = PROJECT_ROOT / "MEBOW"
MEBOW_WEIGHT = MEBOW_ROOT / "models" / "model_hboe.pth"

print(f"  DATA_RAW: {DATA_RAW}")
print(f"  DATA_OUT: {DATA_OUT}")
print(f"  COCO_ANN_DIR: {COCO_ANN_DIR}")
print(f"  MEBOW_ROOT: {MEBOW_ROOT}")

# ----- GPU確認 -----
import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

## 2. ディレクトリ構造の作成

YOLO学習に必要なディレクトリ構造を作成します。

In [ ]:
# ============================================================================
# ディレクトリ構造の作成
# ============================================================================
# 
# seg-person-dir/
#   data/
#     annotations/                    # COCOアノテーション
#     raw/                            # COCO画像DL先（ゲート通過画像のみ）
#       train/
#       test/
#       val/
#     dataset_frontback_yoloseg/      # YOLO形式データセット（出力）
#       images/train, val, test/
#       labels/train, val, test/
#       reports/                      # CSVレポート
#       data.yaml


dirs_to_create = [
    COCO_ANN_DIR,
    DATA_RAW / "train",
    DATA_RAW / "test",
    DATA_RAW / "val",
    DATA_OUT / "images" / "train",
    DATA_OUT / "images" / "test",
    DATA_OUT / "images" / "val",
    DATA_OUT / "labels" / "train",
    DATA_OUT / "labels" / "test",
    DATA_OUT / "labels" / "val",
    PROJECT_ROOT / "runs",
]

for d in dirs_to_create:
    d.mkdir(parents=True, exist_ok=True)
    print(f"OK {d}")

print("\nDirectory structure created.")

## 3. COCOアノテーション読み込み → キーポイントフィルタ → Split作成

COCOアノテーションから person を含む画像IDを抽出し、**キーポイントゲートで品質フィルタ**した上で split（train/test/val）を作成します。

### 手順
1. instances + keypoints アノテーションをロード
2. person を含む画像IDに対してキーポイントゲート判定（**アノテーション参照のみ**で、画像のダウンロードはまだ不要）
3. ゲート通過IDをシャッフルし、`N_TRAIN` / `N_TEST` 分だけ抽出（不足する場合は存在分）
4. `val` は `val2017` のゲート通過IDを全数使用

### キーポイントゲート条件
- 必須：`left_shoulder`(5), `right_shoulder`(6), `left_knee`(13), `right_knee`(14)
- 可視性：`v >= 1`（ラベル付き。遮蔽を含む）
- 画像内に1人でも条件を満たす歩行者がいれば通過

### 前提（必要ファイル）
- `data/annotations/instances_train2017.json`
- `data/annotations/instances_val2017.json`
- `data/annotations/person_keypoints_train2017.json`
- `data/annotations/person_keypoints_val2017.json`

未取得の場合：
```text
http://images.cocodataset.org/annotations/annotations_trainval2017.zip
```

In [ ]:
# ============================================================================
# COCOアノテーション読み込み → キーポイントフィルタ → Split作成
# ============================================================================
from pycocotools.coco import COCO as COCOAPI

# ----- 設定 -----
N_TRAIN = 20000   # train画像数（フィルタ後）
N_TEST  = 3000    # test画像数（フィルタ後）
SEED = 42

# ----- キーポイントゲート設定 -----
# どの部位が見えていれば、フィルタリングを通過とみなすか
REQUIRED_KP_INDICES = [5, 6, 13, 14]  # 両肩, 両ひざ
# どの程度映っていれば、フィルタリング通過とみなすか（v=0：未ラベル、v=1：ラベル付き（遮蔽含む）, v=2：可視）
KP_VIS_THRESHOLD = 1  # v>=1

def is_valid_person(ann):
    """キーポイント条件を満たす歩行者か判定"""
    kps = ann.get("keypoints", None)
    if kps is None or len(kps) < 51:
        return False
    for idx in REQUIRED_KP_INDICES:
        if kps[idx * 3 + 2] < KP_VIS_THRESHOLD:
            return False
    return True

def passes_keypoint_gate(img_id, coco_kp, person_cat_id):
    """画像レベルゲート: 1人でも全身が映っていれば通過"""
    ann_ids = coco_kp.getAnnIds(imgIds=[img_id], catIds=[person_cat_id], iscrowd=0)
    anns = coco_kp.loadAnns(ann_ids)
    return any(is_valid_person(a) for a in anns)

# ----- アノテーション存在チェック -----
assert ANN_TRAIN.exists(), f"Not found: {ANN_TRAIN}"
assert ANN_VAL.exists(), f"Not found: {ANN_VAL}"
assert KP_TRAIN.exists(), f"Not found: {KP_TRAIN}\nDL: http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
assert KP_VAL.exists(), f"Not found: {KP_VAL}"
print(f"OK: {ANN_TRAIN.name}, {ANN_VAL.name}, {KP_TRAIN.name}, {KP_VAL.name}")

# ----- COCOロード -----
coco_train = COCOAPI(str(ANN_TRAIN))
coco_val   = COCOAPI(str(ANN_VAL))
coco_kp_train = COCOAPI(str(KP_TRAIN))
coco_kp_val   = COCOAPI(str(KP_VAL))

person_id_train = coco_train.getCatIds(catNms=["person"])[0]
person_id_val   = coco_val.getCatIds(catNms=["person"])[0]

# ----- train2017: 全person画像 → キーポイントフィルタ -----
all_person_ids = coco_train.getImgIds(catIds=[person_id_train])
print(f"\ntrain2017 person images: {len(all_person_ids)}")

clean_person_ids = [
    img_id for img_id in all_person_ids
    if passes_keypoint_gate(img_id, coco_kp_train, person_id_train)
]
print(f"  → gate passed: {len(clean_person_ids)}  (filtered out: {len(all_person_ids) - len(clean_person_ids)})")

# サンプリング（フィルタ後のIDからN_TRAIN + N_TESTを取得）
assert len(clean_person_ids) >= N_TRAIN + N_TEST, \
    f"Not enough clean images: {len(clean_person_ids)} < {N_TRAIN + N_TEST}"

random.seed(SEED)
random.shuffle(clean_person_ids)
train_ids = clean_person_ids[:N_TRAIN]
test_ids  = clean_person_ids[N_TRAIN:N_TRAIN + N_TEST]

# ----- val2017: 全person画像 → キーポイントフィルタ -----
all_val_person_ids = coco_val.getImgIds(catIds=[person_id_val])
print(f"\nval2017 person images: {len(all_val_person_ids)}")

val_ids = [
    img_id for img_id in all_val_person_ids
    if passes_keypoint_gate(img_id, coco_kp_val, person_id_val)
]
print(f"  → gate passed: {len(val_ids)}  (filtered out: {len(all_val_person_ids) - len(val_ids)})")

# ----- 結果 -----
print(f"\nSplit sizes (post-filter):")
print(f"  train: {len(train_ids)}")
print(f"  test:  {len(test_ids)}")
print(f"  val:   {len(val_ids)}")

## 4. 画像ダウンロード

`coco_url` を使って必要な画像をダウンロードします。

- 並列DL（ThreadPoolExecutor）で高速化
- 既存ファイルはスキップ

In [ ]:
# ============================================================================
# 画像ダウンロード
# ============================================================================
import urllib.request
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def download_image(url: str, dest: Path) -> bool:
    """1枚の画像をダウンロード（既存ならスキップ）"""
    if dest.exists():
        return True
    try:
        urllib.request.urlretrieve(url, dest)
        return True
    except Exception as e:
        print(f"Failed: {url} -> {e}")
        return False

def download_images_parallel(coco, img_ids: list, out_dir: Path, max_workers: int = 8):
    """並列で画像をダウンロード"""
    out_dir.mkdir(parents=True, exist_ok=True)
    
    tasks = []
    for img_id in img_ids:
        info = coco.loadImgs(img_id)[0]
        url = info["coco_url"]
        dest = out_dir / info["file_name"]
        tasks.append((url, dest))
    
    success = 0
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_image, url, dest): (url, dest) for url, dest in tasks}
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Download -> {out_dir.name}"):
            if future.result():
                success += 1
    
    print(f"  {success}/{len(tasks)} downloaded to {out_dir}")
    return success

# ----- ダウンロード実行 -----
print("Downloading train images...")
download_images_parallel(coco_train, train_ids, DATA_RAW / "train")

print("Downloading test images...")
download_images_parallel(coco_train, test_ids, DATA_RAW / "test")

print("Downloading val images...")
download_images_parallel(coco_val, val_ids, DATA_RAW / "val")

print("\nDownload complete!")

## 5. MEBOWモデルのロードと角度推定

MEBOWモデルで歩行者cropから向き（角度）を推定し、SAME/OPSクラスに分類します。

**前提:** `MEBOW/models/model_hboe.pth` が必要です

In [ ]:
# ============================================================================
# MEBOWモデルのロードと角度推定
# ============================================================================
import cv2
import numpy as np
import torchvision.transforms as transforms
from types import SimpleNamespace

# ----- MEBOWパス設定 -----
MEBOW_LIB = (MEBOW_ROOT / "lib").resolve()
if str(MEBOW_LIB) not in sys.path:
    sys.path.insert(0, str(MEBOW_LIB))

from config import cfg as mebow_cfg
from config import update_config as mebow_update_config
import models as mebow_models

# ----- 前処理 -----
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
mebow_transform = transforms.Compose([transforms.ToTensor(), normalize])

def load_mebow_model(
    cfg_path="experiments/coco/segm-4_lr1e-3.yaml",
    weight_path="models/model_hboe.pth",
):
    """MEBOWモデルをロード"""
    cfg_path = (MEBOW_ROOT / cfg_path).resolve()
    weight_path = (MEBOW_ROOT / weight_path).resolve()
    
    assert weight_path.exists(), f"MEBOW weight not found: {weight_path}"
    
    args = SimpleNamespace(cfg=str(cfg_path), opts=[], modelDir="", logDir="", dataDir="")
    mebow_update_config(mebow_cfg, args)
    
    model = mebow_models.pose_hrnet.get_pose_net(mebow_cfg, is_train=False)
    state = torch.load(weight_path, map_location=DEVICE)
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    model.load_state_dict(state, strict=False)
    model = model.to(DEVICE).eval()
    
    width, height = mebow_cfg.MODEL.IMAGE_SIZE
    return model, (int(width), int(height))

# ----- モデルロード -----
mebow_model, MEBOW_INPUT_SIZE = load_mebow_model()
print(f"MEBOW loaded. Input size: {MEBOW_INPUT_SIZE}, Device: {DEVICE}")

@torch.inference_mode()
def mebow_predict_angles_from_crops(crops_rgb: list) -> list:
    """
    複数のcrop画像（RGB）から角度を推定
    
    Args:
        crops_rgb: RGB画像のリスト
    Returns:
        角度のリスト（0-355度、5度刻み）
    """
    if len(crops_rgb) == 0:
        return []
    
    xs = []
    for crop in crops_rgb:
        crop_rs = cv2.resize(crop, MEBOW_INPUT_SIZE, interpolation=cv2.INTER_LINEAR)
        xs.append(mebow_transform(crop_rs))
    
    x = torch.stack(xs).to(DEVICE)
    _, hoe = mebow_model(x)  # (N, 72) - 5度刻み72クラス
    idx = torch.argmax(hoe, dim=1)
    return (idx * 5).tolist()

# ----- クラス分類関数 -----
SAME = 0
OPS = 1
TH_SAME = 45  # same-dirの半幅（0°±45°）

def angle_to_class(angle_deg: float) -> int:
    """
    MEBOW角度をSAME/OPSに分類
    
    - SAME (0): 0°±45° or 360°-45°～360°
    - OPS (1): それ以外
    """
    a = angle_deg % 360
    if a <= TH_SAME or a >= (360 - TH_SAME):
        return SAME
    return OPS

print("angle_to_class defined. (SAME=0, OPS=1, threshold=45deg)")

## 6. MEBOW向き推定 + YOLO-Seg変換＆エクスポート

ゲート通過済みの画像に対して、MEBOW向き推定とYOLO-Seg形式ラベル生成を行います。

**処理内容:**
1. 画像を読み込み、全person の bbox を crop（サイズフィルタ: 32x64px以上）
2. MEBOW で向き角度を推定 → SAME/OPS クラスに分類
3. COCOセグメンテーション → YOLO正規化ポリゴンに変換
4. ラベル保存 + 画像コピー
5. CSVレポート出力（`reports/{split}_export_report.csv`）

In [ ]:
# ============================================================================
# MEBOW向き推定 + YOLO-Seg変換 — 関数定義
# ============================================================================
from pycocotools import mask as coco_mask
import csv

# 再実行時に過去出力を自動削除
REMOVE_STALE_OUTPUTS = True
REPORT_DIR = DATA_OUT / "reports"

# ----- COCO → ポリゴン変換 -----

def coco_ann_to_polygons(ann, H, W):
    """COCOアノテーションからポリゴン座標を抽出"""
    seg = ann.get("segmentation", None)
    if seg is None:
        return []

    if isinstance(seg, list):
        polys = []
        for s in seg:
            pts = np.array(s, dtype=np.float32).reshape(-1, 2)
            if len(pts) >= 3:
                polys.append(pts)
        return polys

    if isinstance(seg, dict):
        rle = coco_mask.frPyObjects(seg, H, W) if isinstance(seg.get("counts"), list) else seg
        m = coco_mask.decode(rle)
        if m.ndim == 3:
            m = m[:, :, 0]
        m = (m > 0).astype(np.uint8) * 255
        cnts, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return [c.reshape(-1, 2).astype(np.float32) for c in cnts if len(c) >= 3]

    return []


def normalize_poly(poly_xy, H, W):
    """ポリゴン座標を0-1に正規化"""
    p = poly_xy.copy()
    p[:, 0] = np.clip(p[:, 0] / W, 0, 1)
    p[:, 1] = np.clip(p[:, 1] / H, 0, 1)
    return p


# ----- エクスポート関数 -----

EXPORT_FIELDS = ["file_name", "img_id", "status", "reason", "n_persons", "classes"]

def export_split(split: str, coco, img_dir: Path, img_ids: list, person_cat_id: int):
    """
    ゲート通過済み画像をYOLO-Seg形式でエクスポート

    Returns:
        records: 各画像の処理結果レコード
    """
    out_images = DATA_OUT / "images" / split
    out_labels = DATA_OUT / "labels" / split
    out_images.mkdir(parents=True, exist_ok=True)
    out_labels.mkdir(parents=True, exist_ok=True)

    def remove_outputs(file_name: str):
        if not REMOVE_STALE_OUTPUTS:
            return
        lbl = out_labels / f"{Path(file_name).stem}.txt"
        img = out_images / file_name
        if lbl.exists(): lbl.unlink()
        if img.exists(): img.unlink()

    success_count = 0
    skip_count = 0
    records = []
    class_names = {SAME: "SAME", OPS: "OPS"}

    for img_id in tqdm(img_ids, desc=f"Export {split}"):
        info = coco.loadImgs(img_id)[0]
        fn = info["file_name"]
        src_img = img_dir / fn

        img_bgr = cv2.imread(str(src_img))
        if img_bgr is None:
            remove_outputs(fn)
            skip_count += 1
            records.append({"file_name": fn, "img_id": img_id, "status": "skipped",
                            "reason": "image_not_found", "n_persons": 0, "classes": ""})
            continue

        H, W = img_bgr.shape[:2]

        # 全person を crop（サイズフィルタ）
        ann_ids = coco.getAnnIds(imgIds=[img_id], catIds=[person_cat_id], iscrowd=0)
        anns = coco.loadAnns(ann_ids)
        crops_rgb, valid_anns = [], []

        for ann in anns:
            x, y, w, h = ann["bbox"]
            if w * h < 32 * 64:
                continue
            x0, y0 = max(int(x), 0), max(int(y), 0)
            x1, y1 = min(int(x + w), W), min(int(y + h), H)
            crop = img_bgr[y0:y1, x0:x1]
            if crop.size == 0:
                continue
            crops_rgb.append(crop[:, :, ::-1])
            valid_anns.append(ann)

        if not crops_rgb:
            remove_outputs(fn)
            skip_count += 1
            records.append({"file_name": fn, "img_id": img_id, "status": "skipped",
                            "reason": "no_valid_crops", "n_persons": len(anns), "classes": ""})
            continue

        # MEBOW角度推定
        angles = mebow_predict_angles_from_crops(crops_rgb)

        # ラベル生成
        lines, line_classes = [], []
        for ann, ang in zip(valid_anns, angles):
            cls = angle_to_class(ang)
            polys = coco_ann_to_polygons(ann, H, W)
            if not polys:
                continue
            poly = max(polys, key=lambda q: abs(cv2.contourArea(q.astype(np.float32))))
            eps = 0.002 * (H + W)
            poly2 = cv2.approxPolyDP(poly.astype(np.float32), eps, True).reshape(-1, 2)
            if len(poly2) < 3:
                continue
            pn = normalize_poly(poly2, H, W).flatten()
            lines.append(str(cls) + " " + " ".join([f"{v:.6f}" for v in pn.tolist()]))
            line_classes.append(class_names[cls])

        if not lines:
            remove_outputs(fn)
            skip_count += 1
            records.append({"file_name": fn, "img_id": img_id, "status": "skipped",
                            "reason": "no_valid_labels", "n_persons": len(valid_anns), "classes": ""})
            continue

        # ラベル保存
        label_path = out_labels / f"{Path(fn).stem}.txt"
        label_path.write_text("\n".join(lines), encoding="utf-8")

        # 画像コピー
        dst_img = out_images / fn
        if not dst_img.exists():
            shutil.copy2(src_img, dst_img)

        success_count += 1
        records.append({"file_name": fn, "img_id": img_id, "status": "kept",
                        "reason": "", "n_persons": len(lines), "classes": ",".join(line_classes)})

    # CSVレポート保存
    REPORT_DIR.mkdir(parents=True, exist_ok=True)
    report_path = REPORT_DIR / f"{split}_export_report.csv"
    with open(report_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=EXPORT_FIELDS)
        writer.writeheader()
        writer.writerows(records)

    print(f"  {split}: {success_count} kept, {skip_count} skipped  |  Report: {report_path}")
    return records


print("export_split defined.")

## 6.1 エクスポート実行

`export_split()` を `train` / `test` / `val` それぞれに対して実行し、以下を出力します。

- 画像：`data/dataset_frontback_yoloseg/images/{split}/`
- ラベル：`data/dataset_frontback_yoloseg/labels/{split}/`（YOLO-Seg形式）
- レポート：`data/dataset_frontback_yoloseg/reports/{split}_export_report.csv`

In [ ]:
# ============================================================================
# エクスポート実行
# ============================================================================
from collections import Counter

print("Exporting train...")
train_records = export_split("train", coco_train, DATA_RAW / "train", train_ids, person_id_train)

print("\nExporting test...")
test_records = export_split("test", coco_train, DATA_RAW / "test", test_ids, person_id_train)

print("\nExporting val...")
val_records = export_split("val", coco_val, DATA_RAW / "val", val_ids, person_id_val)

# ----- サマリー -----
print(f"\n{'='*60}")
print("Export Summary")
print(f"{'='*60}")
for name, recs in [("train", train_records), ("test", test_records), ("val", val_records)]:
    kept = sum(1 for r in recs if r["status"] == "kept")
    skipped = sum(1 for r in recs if r["status"] == "skipped")
    print(f"  {name}: {kept} kept, {skipped} skipped")

print(f"\nクラス分布 (person単位):")
for name, recs in [("train", train_records), ("test", test_records), ("val", val_records)]:
    all_cls = []
    for r in recs:
        if r["classes"]:
            all_cls.extend(r["classes"].split(","))
    cls_counts = Counter(all_cls)
    parts = [f"{c}:{n}" for c, n in cls_counts.most_common()]
    print(f"  {name}: {', '.join(parts)}")
print(f"{'='*60}")

## 7. data.yaml 生成

YOLO学習に必要な設定ファイルを生成します。

In [ ]:
# ============================================================================
# data.yaml 生成
# ============================================================================
data_yaml_content = f"""path: {DATA_OUT.resolve()}
train: images/train
val: images/val
test: images/test
names:
  0: same-dir-person
  1: ops-dir-person
"""

data_yaml_path = DATA_OUT / "data.yaml"
data_yaml_path.write_text(data_yaml_content, encoding="utf-8")

print("data.yaml created:")
print("-" * 40)
print(data_yaml_path.read_text())
print("-" * 40)

## 8. データセット確認

生成されたデータセットのファイル数を確認します。

In [ ]:
# ============================================================================
# データセット確認
# ============================================================================
print("=" * 50)
print("Dataset Summary")
print("=" * 50)

for split in ["train", "test", "val"]:
    img_dir = DATA_OUT / "images" / split
    lbl_dir = DATA_OUT / "labels" / split

    n_img = len(list(img_dir.glob("*.jpg")))
    n_lbl = len(list(lbl_dir.glob("*.txt")))

    print(f"  {split:5s}: {n_img:5d} images, {n_lbl:5d} labels")

print("=" * 50)
print(f"data.yaml: {DATA_OUT / 'data.yaml'}")
print("=" * 50)